
# ViEWS 3 constituent models 
## FCDO project, cm level


This notebook trains a set of regression models for use in a predicting fatalities ensemble

New version May 2022 using a model list that can be stored as csv rather than a pickle

The notebook does the following: 
1. Retrieves data through querysets and stores in DataSets, a list of dictionaries
2. Specifies the metadata of a number of models, stores in ModelList, a list of dictionaries
3. Trains the models in ModelList, stores the trained objects in model storage and prediction storage
4. Saves part of ModelList as csv and the rest as pickles


## Importing modules

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier

from lightgbm import LGBMClassifier, LGBMRegressor

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage
from views_runs.storage import store, retrieve, list, fetch_metadata

from views_forecasts.extensions import *

# Other packages
import pickle as pkl

# Packages from Predicting Fatalies repository

#from HurdleRegression import * # Built on script from Geoff Hurdock: https://geoffruddock.com/building-a-hurdle-regression-estimator-in-scikit-learn/
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
from FetchData import FetchData, RetrieveFromList
from ViewsEstimators import *


## Common parameters

In [ ]:
#!conda list | grep views

In [ ]:
# To do:
# find out why and where missingness occurs

In [ ]:
# Common parameters:
dev_id = 'Fatalities001'
run_id = 'Fat_devel_v7'

# Generating a new run if necessary

try:
    ViewsMetadata().new_run(name=run_id,description='Developing the fatalities model for FCDO',min_month=1,max_month=999)
except KeyError:
    if 'devel' not in run_id:
        warnings.warn('You are overwriting a production system')

RerunQuerysets = True

FutureStart = 506
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for
fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,3,6,12,36]
#fi_steps = [1,3,6,12,36]

# Specifying partitions
calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = '/Users/havardhegre/Dropbox (ViEWS)/ViEWS'

# Retrieve data

In [ ]:
# Specifying querysets
# Rerun if querysets have 
if RerunQuerysets:
    import cm_querysets

In [ ]:

Datasets = FetchData(dev_id)

# Generating predictions
Using the new partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome, but also one model with ged_sb_best to see whether that improves calibration on its own.

In [ ]:
Datasets[0]['df'].head()

In [ ]:
from views_runs import ModelMetadata 
help(ModelMetadata)

## Checking missingness and infinity values

In [ ]:
N=51
df = Datasets[0]['df']
for col in df.iloc[: , :N].columns:
    print(col,len(df[col]), 'missing:', df[col].isnull().sum(), 'infinity:', np.isinf(df).values.sum())


## Identify early stopping parameter

In [ ]:
np.log(.10/.9)

In [ ]:
es_steps = [3,6,24]




xgb2 = XGBRegressor(n_estimators=500, tree_method='hist', max_depth=5). self.stopped_model = xgb2.fit(X=X, y=y,
                             eval_set=[(X, y), (X_cal, y_cal)],
                             verbose = True, early_stopping_rounds=10)

# Specify models in ensemble

In [ ]:
# Early stopping model revision: 

#(consider varying n_estimators for ensemble: 80, 100, 120)

model = {
    'modelname':  'fat_conflicthistory_xgb',
    'algorithm': XGBRegressor(n_estimators=100, learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}

In [ ]:
# The ModelList is a list of dictionaries that define a range of models for the project

# To do: Include the preprocessing function as item in dictionary, to be able to run the PCA models.

ModelList = []
nj=12

model = {
    'modelname':     'fat_baseline_rf',
    'algorithm':     XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':        "ln_ged_sb_dep",
    'data_train':    'baseline',
    'queryset':      'hh_fatalities_ged_ln_ultrashort',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_srf',
    'algorithm': RandomForestRegressor(n_estimators=200, n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

# Model: GED logged dependent variable, logged conflict history variables, gradient boosting
model = {
    'modelname': 'fat_conflicthistory_gbm',
    'algorithm': GradientBoostingRegressor(), 
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)       
    
# Conflict history hurdle
 # Model: GED logged dependent variable, hurdle/random forest with default settings
model = {
    'modelname': 'fat_conflicthistory_hurdle_rf',
    'algorithm': HurdleRegression(clf_name = 'RFClassifier', reg_name = 'RFRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_hurdle_xgb',
    'algorithm': HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_hurdle_lgb',
    'algorithm': HurdleRegression(clf_name = 'LGBMClassifier', reg_name = 'LGBMRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_histgbm',
    'algorithm': HistGradientBoostingRegressor(max_iter=200),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_conflicthistory_xgb',
    'algorithm': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_conflicthistory_xgb_gamma',
    'algorithm': XGBRegressor(n_estimators=200,tree_method='hist', objective='reg:gamma',n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
#ModelList.append(model)

model = {
    'modelname':  'fat_conflicthistory_xgb_poisson',
    'algorithm': XGBRegressor(n_estimators=200,tree_method='hist', objective='count:poisson',n_jobs=nj),
    'depvar': "ged_sb_dep",
    'data_train':    'conflict_nolog',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_conflicthistory_lgb',
    'algorithm': LGBMRegressor(n_estimators=100,num_threads=8),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_conflicthistory_long_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflictlong_ln',
    'queryset':  "hh_fatalities_ged_acled_ln",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_long_xgb',
    'algorithm': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflictlong_ln',
    'queryset': "hh_fatalities_ged_acled_ln",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_vdem_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'vdem_short',
    'queryset': "hh_fatalities_vdem_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_vdem_xgb',
    'algorithm': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'vdem_short',
    'queryset':  "hh_fatalities_vdem_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)


model = {
    'modelname':  'fat_vdem_hurdle_xgb',
    'algorithm': HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'vdem_short',
    'queryset':  "hh_fatalities_vdem_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_wdi_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'wdi_short',
    'queryset':  "hh_fatalities_wdi_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)


model = {
    'modelname':  'fat_wdi_xgb',
    'algorithm': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'wdi_short',
    'queryset': "hh_fatalities_wdi_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)



model = {
    'modelname':  'fat_wdi_hurdle_xgb',
    'algorithm': HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'wdi_short',
    'queryset':  "hh_fatalities_wdi_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_topics_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'topics',
    'queryset':   "hh_topic_model_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_topics_histgbm',
    'algorithm': HistGradientBoostingRegressor(max_iter=200),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'topics',
    'queryset':   "hh_topic_model_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_topics_xgb',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'topics',
    'queryset':   "hh_topic_model_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_topics_hurdle_xgb',
    'algorithm': HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'topics',
    'queryset':   "hh_topic_model_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_prs_rf',
    'algorithm':  XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'prs',
    'queryset':   'hh_prs',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_prs_xgb',
    'algorithm':  XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'prs',
    'queryset':   'hh_prs',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_prs_hurdle_xgb',
    'algorithm':  HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'prs',
    'queryset':   'hh_prs',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_broad_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'broad',
    'queryset':   'hh_broad',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_broad_xgb',
    'algorithm':  XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'broad',
    'queryset':   'hh_broad',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_broad_hurdle_xgb',
    'algorithm':  HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'broad',
    'queryset':   'hh_broad',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_greatest_hits_rf',
    'algorithm':  XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'gh',
    'queryset':   'hh_greatest_hits',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_greatest_hits_xgb',
    'algorithm':  XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'gh',
    'queryset':   'hh_greatest_hits',
    'preprocessing': 'float_it',
}
ModelList.append(model)


model = {
    'modelname':  'fat_greatest_hits_hurdle_xgb',
    'algorithm':  HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'gh',
    'queryset':   'hh_greatest_hits',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_greatest_hits_lgb',
    'algorithm':  LGBMRegressor(n_estimators=100,num_threads=8),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'gh',
    'queryset':   'hh_greatest_hits',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_hh20_srf',
    'algorithm': RandomForestRegressor(n_estimators=200, n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_hh20_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_hh20_gbm',
    'algorithm': GradientBoostingRegressor(), 
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)       

model = {
    'modelname': 'fat_hh20_hurdle_rf',
    'algorithm': HurdleRegression(clf_name = 'RFClassifier', reg_name = 'RFRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_hh20_hurdle_xgb',
    'algorithm': HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)


model = {
    'modelname': 'fat_hh20_hurdle_lgb',
    'algorithm': HurdleRegression(clf_name = 'LGBMClassifier', reg_name = 'LGBMRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_hh20_histgbm',
    'algorithm': HistGradientBoostingRegressor(max_iter=200),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_hh20_xgb',
    'algorithm': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname' :      'fat_hh20_lgb',
    'algorithm' :      LGBMRegressor(n_estimators=200,num_threads=8),
    'depvar':          "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset':        'hh_20_features',
    'preprocessing':   'float_it',
}
ModelList.append(model)

# PCA models: need to implement a PCA preprocessing function first.
model = {
    'modelname':      'fat_all_pca3_xgb',
    'algorithm':      XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar':         "ln_ged_sb_dep",
    'data_train':     'pca_all',
    'queryset':      'hh_all_features',
    'preprocessing': 'pca_it',
}
ModelList.append(model)

# PCA models: need to implement a PCA preprocessing function first.
model = {
    'modelname':  'fat_all_pca3_lgb',
    'algorithm': LGBMRegressor(n_estimators=100,num_threads=12),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'pca_all',
    'queryset': 'hh_all_features',
    'preprocessing': 'pca_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_topics_pca3_xgb',
    'algorithm': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'pca_topics',
    'queryset': 'hh_topics_short',
    'preprocessing': 'pca_it',
}
ModelList.append(model)



model = {
    'modelname':  'fat_topics_pca3_lgb',
    'algorithm': LGBMRegressor(n_estimators=200,num_threads=12),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'pca_topics',
    'queryset': 'hh_topics_short',
    'preprocessing': 'pca_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_vdem_pca3_lgb',
    'algorithm': LGBMRegressor(n_estimators=200,num_threads=12),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'pca_vdem',
    'queryset': 'hh_vdem_short',
    'preprocessing': 'pca_it',
}
ModelList.append(model)

model = {
    'modelname':      'fat_wdi_pca3_lgb',
    'algorithm':      LGBMRegressor(n_estimators=200,num_threads=12),
    'depvar':         "ln_ged_sb_dep",
    'data_train':     'pca_wdi',
    'queryset':       'hh_wdi_short',
    'preprocessing':  'pca_it',
}
ModelList.append(model)

i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

In [ ]:
# MERF
if False:
    from merf import MERF
    merf = MERF()
    merf.fit(X_train, Z_train, clusters_train, y_train)
    y_hat = merf.predict(X_test, Z_test, clusters_test)

    model = {
        'modelname': 'fat_baseline_merf',
        'algorithm': (n_estimators=200, n_jobs=nj),
        'depvar': "ln_ged_sb_dep",
        'queryset': 'hh_fatalities_ged_ln_ultrashort',
    }

In [ ]:
from views_runs.run_result import RunResult
help(RunResult)

In [ ]:
for model in ModelList[0:5]:
    print(model['modelname'],model['RunResult'].metadata)
    print(model['modelname'],model['RunResult'].data.head())

In [ ]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'cm'
includeFuture = True

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult

i = 0
for model in ModelList[0:1]:
    force_retrain = False
    modelstore = storage.Storage()
    ct = datetime.now()
    print(i, model['modelname'])
    print('Calibration partition', ct)
    model['Algorithm_text'] = str(model['algorithm'])
    model['RunResult_calib'] = RunResult.retrain_or_retrieve(
            retrain            = force_retrain,
            store              = modelstore,
            partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
            stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
            dataset            = RetrieveFromList(Datasets,model['data_train']),
            queryset_name      = model['queryset'],
            partition_name     = "calib",
            timespan_name      = "train",
            storage_name       = model['modelname'] + '_calib',
            author_name        = "HH",
    )

    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
    ct = datetime.now()
    print('Trying to retrieve predictions', ct)
    try:
        predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
    except KeyError:
        print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
        predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
        predictions_calib.forecasts.set_run(run_id)
        predictions_calib.forecasts.to_store(name=model['predstore_calib'])

    ct = datetime.now()
    print('Test partition', ct)
    modelstore = storage.Storage()
    model['RunResult_test'] = RunResult.retrain_or_retrieve(
            retrain            = force_retrain,
            store              = modelstore,
            partitioner        = DataPartitioner({"test":test_partitioner_dict}),
            stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
            dataset            = RetrieveFromList(Datasets,model['data_train']),
            queryset_name      = model['queryset'],
            partition_name     = "test",
            timespan_name      = "train",
            storage_name       = model['modelname'] + '_test',
            author_name        = "HH",
    )
    ct = datetime.now()
    print('Trying to retrieve predictions', ct)
    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
    try:
        predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
    except KeyError:
        print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
        predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
        predictions_test.forecasts.set_run(run_id)
        predictions_test.forecasts.to_store(name=model['predstore_test'])
    # Predictions for true future
    if includeFuture:
        ct = datetime.now()
        print('Future', ct)
        modelstore = storage.Storage()
        model['RunResult_future'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_future',
                author_name        = "HH",
        )
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
        model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(FutureStart)
        try:
            predictions_future = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_future'])
        except KeyError:
            print(model['predstore_future'], ', run', run_id, 'does not exist, predicting')
            predictions_future = model['RunResult_future'].run.future_point_predict(FutureStart,model['RunResult_future'].data)
            predictions_future.forecasts.set_run(run_id)
            predictions_future.forecasts.to_store(name=model['predstore_future'])  
    model['algorithm'] = []
    i = i + 1

print('All done')

In [ ]:
# Exploring the future predictions


predictions_test.xs(246,level=1).tail()

## Notes on training time for the various algorithms:

In [ ]:
#These are calculated in minutes for the hh20 feature set (with about 40 features), for all 36 steps, calibration (c) and test (t) partitions, also include generating predictions, and are approximate:

#nj=12 (number of threads)
#scikit random forest:        21:13 (c), 26:20 (t) RandomForestRegressor(n_estimators=200, n_jobs=nj)
#XGB random forest:           06:02 (c), 07:51 (t) XGBRFRegressor(n_estimators=300,n_jobs=nj)
#scikit gbm:                  13:59 (c), 15:55 (t) GradientBoostingRegressor(), 
#scikit hurdle random forest: 07:32 (c), 09:49 (t) For both clf and reg: (n_estimators=200, n_jobs=nj)
#XGB hurdle xgb:              01:26 (c), 01:32 (t) For both clf and reg:                n_estimators=200,tree_method='hist',n_jobs=nj)
#scikit histgbm:              01:17 (c), 01:20 (t) HistGradientBoostingRegressor(max_iter=200)
#XGB xgb:                     01:00 (c), 01:04 (t) XGBRegressor(n_estimators=200,tree_method='hist',n_jobs=nj)
#lightgbm gbm:                00:25 (c), --    (t) LGBMRegressor(n_estimators=100,num_threads=8)

# Various helper functions and tools....

In [ ]:
# Retrieve current partition
model['run_calib']._partitioner.partitions.dict()


In [ ]:
!conda list | grep views-forecasts

# Retrieving external forecasts

In [ ]:
# Retrieve David's Markov models
# To do: rewrite the model dictionary to the new, slimmer version.
DRList = []


model = {
    'modelname':   'fat_hh20_Markov_glm',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}
ModelList.append(model)
DRList.append(model)

model = {
    'modelname':   'fat_hh20_Markov_rf',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}
ModelList.append(model)
DRList.append(model)



In [ ]:
path = '/Users/havardhegre/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/cm/preds/'

DRList[0]['predictions_file_calib'] = path + 'vmm_glm_hh20_0125_alt_calib.csv'
DRList[0]['predictions_file_test'] = path + 'vmm_glm_hh20_0125_alt_test.csv'
DRList[0]['predictions_file_future'] = path + 'vmm_glm_hh20_506.csv'

DRList[1]['predictions_file_calib'] = path + 'vmm_rf_hh20_0125_alt_calib.csv'
DRList[1]['predictions_file_test'] = path + 'vmm_rf_hh20_0125_alt_test.csv'
DRList[1]['predictions_file_future'] = path + 'vmm_rf_hh20_505.csv'

In [ ]:
# Storing Markov models in central storage
# Retrieving dependent variable
target_calib = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_calib', run=run_id)['ln_ged_sb_dep']
target_test = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_test', run=run_id)['ln_ged_sb_dep']
level = 'cm'
for model in DRList:
    df_calib = pd.read_csv(model['predictions_file_calib'],index_col=['month_id','country_id'])
    df_test = pd.read_csv(model['predictions_file_test'],index_col=['month_id','country_id'])
    df_future = pd.read_csv(model['predictions_file_future'],index_col=['month_id','country_id'])
    df_calib['ln_ged_sb_dep'] = target_calib
    df_test['ln_ged_sb_dep'] = target_test
    df_future['ln_ged_sb_dep'] = np.nan # Empty dependent variable column for consistency/required by prediction storage function
    stored_modelname = level + '_' + model['modelname'] + '_calib'
    df_calib.forecasts.set_run(run_id)
    df_calib.forecasts.to_store(name=stored_modelname, overwrite=True)
    stored_modelname = level + '_' + model['modelname'] + '_test'
    df_test.forecasts.set_run(run_id)
    df_test.forecasts.to_store(name=stored_modelname, overwrite=True)    
    stored_modelname = level + '_' + model['modelname'] + '_f505'
    df_future.forecasts.set_run(run_id)
    df_future.forecasts.to_store(name=stored_modelname, overwrite=True)    
       

In [ ]:
# Removing run result objects before saving
# These should perhaps be discarded immediately above
for model in ModelList[0:-3]:
    model['RunResult_calib'] = []
    model['RunResult_test'] = []

In [ ]:
ModelList

# Save the model list of dictionaries

In [ ]:
ModelList_df = pd.DataFrame.from_dict(ModelList)
localpath = '/Users/havardhegre/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/'

filename = localpath + 'Model_cm_' + model['modelname'] + '_'+ dev_id + '.csv'
ModelList_df.to_csv(filename)

In [ ]:
# The following code might be discarded later

In [ ]:
# Code to pickle them one by one
localpath = '/Users/havardhegre/Pickles/'
for model in ModelList:
    picklename = localpath + 'Model_cm_' + model['modelname'] + '_'+ run_id + '.p'
    print(model['modelname'],picklename)
    pkl.dump(model, open(picklename, "wb" ) )

In [ ]:
df.forecasts.set_run('fat_2022_2')

In [ ]:
#ViewsMetadata().mine().with_name('baseline').fetch()

In [ ]:
# Plot correlations within dataset

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook")
sns.set()
cm = "YlGnBu" 
fig, (ax) = plt.subplots(1, 1, figsize=(20,20))
sns.heatmap(data_pca_all.corr(), cmap=cm)